ESPCN_create_data


In [1]:
import torch 
from torch import nn,optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import h5py
import torch.utils.data as dataf
from torchsummary import summary
import math

In [2]:
upscale = 2
patch_size = 12
label_size = 24
stride_step = 12 # 步长


In [3]:
Train_path = 'Train/'
Test_path = 'Test/Set14/'

In [4]:
def create_data(_path):
    names = os.listdir(_path)
    names = sorted(names)     # 排序
    nums = names.__len__() # 训练图片的数量
    
    data = []
    label = []
    
    for i in range(nums):
        
        name = _path + names[i]
        hr_img = cv2.imread(name,cv2.IMREAD_COLOR)
        hr_img = cv2.cvtColor(hr_img,cv2.COLOR_BGR2YCrCb)
        hr_img = hr_img[:,:,0]
        shape = hr_img.shape  # shape[0] -> 行,也就是高度 shape[1] -> 列,也就是宽度
        #释放 注释 查看 training img gray 图像
        #cv2.imshow('source img',hr_img)
        #cv2.waitKey(0) 
        
        lr_img = cv2.resize(hr_img,(int(shape[1]*patch_size/label_size),int(shape[0]*patch_size/label_size)))
        shape_l = lr_img.shape
        height_loop_num = (shape_l[0]- patch_size)/stride_step
        width_loop_num = (shape_l[1]- patch_size)/stride_step
        
        
        for i in range(int(height_loop_num)):
            for j in range(int(width_loop_num)):
                x = i * stride_step
                y = j * stride_step
                lr_patch = lr_img[x:x+patch_size,y:y+patch_size]
                x_h = math.floor(x*label_size/patch_size)
                y_h = math.floor(y*label_size/patch_size)
                hr_patch = hr_img[x_h:x_h+label_size,y_h:y_h+label_size]
                #cv2.imshow("lr", lr_patch)
                #cv2.imshow("hr", hr_patch)
                #cv2.waitKey(0)
        
                #归一化 (0,1)
                hr_patch = hr_patch.astype(float) / 255.
                lr_patch = lr_patch.astype(float) / 255.
                
                hr = np.zeros((1,label_size,label_size),dtype = np.double)
                lr = np.zeros((1,patch_size,patch_size),dtype = np.double)
                
                hr[0,:,:] = hr_patch
                lr[0,:,:] = lr_patch
                
                data.append(lr)
                label.append(hr)
                
        
    data = np.array(data,dtype=float)
    label = np.array(label,dtype=float)
    return data,label 
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        

In [5]:
data,label = create_data(Train_path)

In [6]:
data.shape,label.shape

((6880, 1, 12, 12), (6880, 1, 24, 24))

In [7]:
x = data.astype(np.float32)
y = label.astype(np.float32)
with h5py.File('ESPCN_train_data.h5', 'w') as h:
        h.create_dataset('data', data=x, shape=x.shape)
        h.create_dataset('label', data=y, shape=y.shape)

In [9]:
test_data,test_label = create_data(Test_path)
test_data.shape,test_label.shape

((4917, 1, 12, 12), (4917, 1, 24, 24))

In [10]:
x = test_data.astype(np.float32)
y = test_label.astype(np.float32)
with h5py.File('ESPCN_test_data.h5', 'w') as h:
        h.create_dataset('data', data=x, shape=x.shape)
        h.create_dataset('label', data=y, shape=y.shape)